# Неделя 6. Vowpal Wabbit

In [1]:
%reload_ext watermark
%watermark -v -m -p numpy,scipy,pandas,matplotlib,statsmodels,sklearn,seaborn,tqdm -g

CPython 3.7.2
IPython 7.2.0

numpy 1.15.4
scipy 1.2.0
pandas 0.23.4
matplotlib 3.0.2
statsmodels 0.9.0
sklearn 0.20.2
seaborn 0.9.0
tqdm 4.31.1

compiler   : Clang 6.0 (clang-600.0.57)
system     : Darwin
release    : 18.5.0
machine    : x86_64
processor  : i386
CPU cores  : 12
interpreter: 64bit
Git hash   : 4c199e294683fb314d8a888496ab119eb1bf765a


In [2]:
from collections import Counter
from pathlib import Path

import numpy
import pandas
from pandas import DataFrame, read_csv
from scipy.sparse import csr_matrix, vstack
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm_notebook

## 2.1. Подготовка данных

Далее посмотрим на Vowpal Wabbit в деле. Правда, в задаче нашего соревнования при бинарной классификации веб-сессий мы разницы не заметим – как по качеству, так и по скорости работы (хотя можете проверить), продемонстрируем всю резвость VW в задаче классификации на 400 классов. Исходные данные все те же самые, но выделено 400 пользователей, и решается задача их идентификации. Скачайте данные [отсюда](https://inclass.kaggle.com/c/identify-me-if-you-can4/data) – файлы `train_sessions_400users.csv` и `test_sessions_400users.csv`.

In [3]:
sites = [f'site{i}' for i in range(1, 11)]

In [4]:
train = read_csv('train_sessions_400users.csv', index_col='session_id')

In [5]:
test = read_csv('test_sessions_400users.csv', index_col='session_id')

In [6]:
train.shape, test.shape, train['user_id'].nunique()

((182793, 21), (46473, 20), 400)

Vowpal Wabbit любит, чтоб метки классов были распределены от 1 до K, где K – число классов в задаче классификации (в нашем случае – 400). Поэтому придется применить `LabelEncoder`, да еще и +1 потом добавить (`LabelEncoder` переводит метки в диапозон от 0 до K-1). Потом надо будет применить обратное преобразование.

In [7]:
y = train['user_id']
y_encoder = LabelEncoder()
y_vw = y_encoder.fit_transform(y) + 1

train = train[sites].fillna(0).astype(numpy.int)
test = test[sites].fillna(0).astype(numpy.int)

Далее будем сравнивать VW с `SGDClassifier` и с логистической регрессией. Всем моделям этим нужна предобработка входных данных. Подготовьте для sklearn-моделей разреженные матрицы, как мы это делали в 5 части:

- объедините обучающиую и тестовую выборки
- выберите только сайты (признаки от `site1` до `site10`)
- замените пропуски на нули (сайты у нас нумеровались с 0)
- переведите в разреженный формат `csr_matrix`
- разбейте обратно на обучающую и тестовую части

In [8]:
X_train_test = pandas.concat([train, test], sort=True)

In [9]:
n_cols = X_train_test.values.max()  # максимальный индекс сайта
X_train_test_sparse = vstack([
    csr_matrix((
        [value for value in counter.values()],  # data: числа визитов
        [key for key, value in counter.items()],  # indices: индексы сайтов
        [0, len(counter)],  # indptr: 0...число ненулевых элементов
    ), shape=(1, n_cols))
    for counter in tqdm_notebook(
        Counter(filter(None, row))  # подсчитываем число визитов каждого сайта в сессии, отфильтровываем нули
        for row in X_train_test.itertuples(index=False)  # ходим по всем сессиям
    )
])

In [10]:
X_train_sparse = X_train_test_sparse[:len(train), :]
X_test_sparse = X_train_test_sparse[len(train):, :]

## 2.2. Валидация по отложенной выборке

Выделим обучающую (70%) и отложенную (30%) части исходной обучающей выборки. Данные не перемешиваем, учитываем, что сессии отсортированы по времени.

In [11]:
share = int(0.7 * train.shape[0])
train_part = train[sites].iloc[:share, :]
valid = train[sites].iloc[share:, :]
X_train_part_sparse = X_train_sparse[:share, :]
X_valid_sparse = X_train_sparse[share:, :]

In [12]:
y_train_part = y[:share]
y_valid = y[share:]
y_train_part_vw = y_vw[:share]
y_valid_vw = y_vw[share:]

Реализуйте функцию, `arrays_to_vw`, переводящую обучающую выборку в формат Vowpal Wabbit.

Вход:

- `X` – матрица NumPy (обучающая выборка)
- `y` (необяз.) - вектор ответов (NumPy). Необязателен, поскольку тестовую матрицу будем обрабатывать этой же функцией
- `train` – флаг, `True` в случае обучающей выборки, `False` – в случае тестовой выборки
- `out_file` – путь к файлу `.vw`, в который будет произведена запись

Детали:

- надо пройтись по всем строкам матрицы `X` и записать через пробел все значения, предварительно добавив вперед нужную метку класса из вектора `y` и знак-разделитель `|`
- в тестовой выборке на месте меток целевого класса можно писать произвольные, допустим, 1

In [13]:
def to_vw(x, y, path):
    with path.open('wt') as fp:
        for row, y_ in zip(x.itertuples(False), y):
            print(f'{y_} | {" ".join(map(str, row))}', file=fp)

Примените написанную функцию к части обучащей выборки `(train_df_part, y_train_part_for_vw)`, к отложенной выборке `(valid_df, y_valid_for_vw)`, ко всей обучающей выборке и ко всей тестовой выборке. Обратите внимание, что на вход наш метод принимает именно матрицы и вектора NumPy.

In [14]:
to_vw(train_part, y_train_part_vw, Path('train_part.vw'))
!head -3 train_part.vw

262 | 23713 23720 23713 23713 23720 23713 23713 23713 23713 23713
82 | 8726 8725 665 8727 45 8725 45 5320 5320 5320
16 | 303 19 303 303 303 303 303 309 303 303


In [15]:
to_vw(valid, y_valid_vw, Path('valid.vw'))
!head -3 valid.vw

4 | 7 923 923 923 11 924 7 924 838 7
160 | 91 198 11 11 302 91 668 311 310 91
312 | 27085 848 118 118 118 118 11 118 118 118


In [16]:
to_vw(test[sites], numpy.ones((len(test),), dtype=numpy.int), Path('test.vw'))
!head -3 test.vw

1 | 9 304 308 307 91 308 312 300 305 309
1 | 838 504 68 11 838 11 838 886 27 305
1 | 190 192 8 189 191 189 190 2375 192 8


In [34]:
to_vw(train[sites], y_vw, Path('train.vw'))
!head -3 train.vw

262 | 23713 23720 23713 23713 23720 23713 23713 23713 23713 23713
82 | 8726 8725 665 8727 45 8725 45 5320 5320 5320
16 | 303 19 303 303 303 303 303 309 303 303


Обучите модель Vowpal Wabbit на выборке `train_part.vw`. Укажите, что решается задача классификации с 400 классами (`--oaa`), сделайте 3 прохода по выборке (`--passes`). Задайте некоторый кэш-файл (`--cache_file`, можно просто указать флаг `-c`), так VW будет быстрее делать все следующие после первого проходы по выборке (прошлый кэш-файл удаляется с помощью аргумента `-k`). Также укажите значение параметра `b=26`. Это число бит, используемых для хэширования, в данном случае нужно больше, чем `18` по умолчанию. Наконец, укажите `random_seed=17`. Остальные параметры пока не меняйте, далее уже в свободном режиме соревнования можете попробовать другие функции потерь.

In [18]:
!vw --version

8.6.1


In [19]:
!vw \
    --random_seed 17 \
    --bit_precision 26 \
    --oaa 400 \
    --data train_part.vw \
    --cache_file train_part.cache \
    --passes 3 \
    --final_regressor train_part_regressor.vw

final_regressor = train_part_regressor.vw
Num weight bits = 26
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
using cache_file = train_part.cache
ignoring text input in favor of cache input
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0      262        1       11
1.000000 1.000000            2            2.0       82      262       11
1.000000 1.000000            4            4.0      241      262       11
1.000000 1.000000            8            8.0      352      262       11
1.000000 1.000000           16           16.0      135       16       11
1.000000 1.000000           32           32.0       71      112       11
0.968750 0.937500           64           64.0      358      231       11
0.976562 0.984375          128          128.0      348      346       11
0.941406 0.906250          256         

Запишите прогнозы на выборке `valid.vw` в `vw_valid_pred.csv`.

In [20]:
!vw \
    --initial_regressor train_part_regressor.vw \
    --testonly \
    --data valid.vw \
    --predictions valid_predictions_vw.csv
!echo
!head -3 valid_predictions_vw.csv

only testing
predictions = valid_predictions_vw.csv
Num weight bits = 26
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = valid.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0        4      188       11
1.000000 1.000000            2            2.0      160      220       11
0.750000 0.500000            4            4.0      143      143       11
0.750000 0.750000            8            8.0      247      247       11
0.687500 0.625000           16           16.0      341       30       11
0.593750 0.500000           32           32.0      237      237       11
0.609375 0.625000           64           64.0      178      178       11
0.640625 0.671875          128          128.0      132      228       11
0.656250 0.671875          256          256.0       14       14       11
0.646484 0.636719

Считайте прогнозы `kaggle_data/vw_valid_pred.csv` из файла и посмотрите на долю правильных ответов на отложенной части.

**Вопрос 1.** Посчитайте долю правильных ответов на отложенной выборке для Vowpal Wabbit, округлите до 3 знаков после запятой.

In [21]:
answer6_1 = accuracy_score(y_valid_vw, read_csv("valid_predictions_vw.csv", header=None))
Path('answer6_1.txt').write_text(f'{answer6_1:.3f}')
!cat answer6_1.txt

0.345

Теперь обучите `SGDClassifier` (3 прохода по выборке, логистическая функция потерь) и `LogisticRegression` на 70% разреженной обучающей выборки – `(X_train_part_sparse, y_train_part)`, сделайте прогноз для отложенной выборки `(X_valid_sparse, y_valid)` и посчитайте доли верных ответов. Логистическая регрессия будет обучаться не быстро (у меня – 4 минуты) – это нормально. Укажите везде `random_state=17, n_jobs=-1`. Для `SGDClassifier` также укажите `max_iter=3`.

**Вопрос 2.** Посчитайте долю правильных ответов на отложенной выборке для SGD, округлите до 3 знаков после запятой.

In [28]:
sgd = SGDClassifier(random_state=17, n_jobs=-1, max_iter=3, tol=None, loss='log')
answer6_2 = accuracy_score(
    y_valid,
    sgd.fit(X_train_part_sparse, y_train_part).predict(X_valid_sparse),
)
Path('answer6_2.txt').write_text(f'{answer6_2:.3f}')
!cat answer6_2.txt

/Users/eigenein/GitHub/mipt-ya-ml-spec/venv/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


0.291

**Вопрос 3.** Посчитайте долю правильных ответов на отложенной выборке для логистической регрессии, округлите до 3 знаков после запятой.

In [29]:
%%time

lr = LogisticRegression(random_state=17, verbose=3, multi_class='multinomial')
answer6_3 = accuracy_score(
    y_valid,
    lr.fit(X_train_part_sparse, y_train_part).predict(X_valid_sparse),
)
Path('answer6_3.txt').write_text(f'{answer6_3:.3f}')
!cat answer6_3.txt
!echo

/Users/eigenein/GitHub/mipt-ya-ml-spec/venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/eigenein/GitHub/mipt-ya-ml-spec/venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/eigenein/GitHub/mipt-ya-ml-spec/venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


[LibLinear]0.363
CPU times: user 19min 8s, sys: 1.87 s, total: 19min 9s
Wall time: 19min 10s


## 2.3. Валидация по тестовой выборке (Public Leaderboard)

Обучите модель VW с теми же параметрами на всей обучающей выборке – `train.vw`.

In [35]:
!vw \
    --random_seed 17 \
    --bit_precision 26 \
    --oaa 400 \
    --data train.vw \
    --cache_file train.cache \
    --passes 3 \
    --final_regressor train_regressor.vw

final_regressor = train_regressor.vw
Num weight bits = 26
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = train.cache
Reading datafile = train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0      262        1       11
1.000000 1.000000            2            2.0       82      262       11
1.000000 1.000000            4            4.0      241      262       11
1.000000 1.000000            8            8.0      352      262       11
1.000000 1.000000           16           16.0      135       16       11
1.000000 1.000000           32           32.0       71      112       11
0.968750 0.937500           64           64.0      358      231       11
0.976562 0.984375          128          128.0      348      346       11
0.941406 0.906250          256          256.0      202      20

Сделайте прогноз для тестовой выборки.

In [36]:
!vw \
    --initial_regressor train_regressor.vw \
    --testonly \
    --data test.vw \
    --predictions test_predictions_vw.csv
!echo
!head -3 test_predictions_vw.csv

only testing
predictions = test_predictions_vw.csv
Num weight bits = 26
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0        1       90       11
1.000000 1.000000            2            2.0        1       21       11
1.000000 1.000000            4            4.0        1      265       11
1.000000 1.000000            8            8.0        1      137       11
1.000000 1.000000           16           16.0        1      273       11
1.000000 1.000000           32           32.0        1      384       11
1.000000 1.000000           64           64.0        1      139       11
1.000000 1.000000          128          128.0        1       85       11
1.000000 1.000000          256          256.0        1       25       11
0.994141 0.988281  

Запишите прогноз в файл, примените обратное преобразование меток (был `LabelEncoder` и потом +1 в меткам) и отправьте решение на Kaggle.

In [41]:
def write_answer(y_test: numpy.ndarray, name: str):
    pandas \
        .DataFrame(
            y_encoder.inverse_transform(y_test - 1),
            index=numpy.arange(1, y_test.shape[0] + 1),
            columns=['user_id'],
        ) \
        .to_csv(name, index_label='session_id', float_format='%f')

In [42]:
write_answer(read_csv('test_predictions_vw.csv', header=None).values.reshape(-1), 'test_predictions_vw_kaggle.csv')
!head -3 test_predictions_vw_kaggle.csv

session_id,user_id
1,224
2,48


**Вопрос 4.** Какова доля правильных ответов на публичной части тестовой выборки (public leaderboard) для Vowpal Wabbit?

In [44]:
Path('answer6_4.txt').write_text('0.18768');

**Вопрос 5.** Какова доля правильных ответов на публичной части тестовой выборки (public leaderboard) для SGD?

In [51]:
pandas \
    .DataFrame(
        sgd.fit(X_train_part_sparse, y_train_part).predict(X_test_sparse),
        index=numpy.arange(1, X_test_sparse.shape[0] + 1),
        columns=['user_id'],
    ) \
    .to_csv('test_predictions_sgd_kaggle.csv', index_label='session_id', float_format='%f')
!head -3 test_predictions_sgd_kaggle.csv

session_id,user_id
1,50
2,149


In [52]:
Path('answer6_5.txt').write_text('0.17855');

**Вопрос 6.** Какова доля правильных ответов на публичной части тестовой выборки (public leaderboard) для логистической регрессии?

In [53]:
pandas \
    .DataFrame(
        lr.fit(X_train_sparse, y).predict(X_test_sparse),
        index=numpy.arange(1, X_test_sparse.shape[0] + 1),
        columns=['user_id'],
    ) \
    .to_csv('test_predictions_lr_kaggle.csv', index_label='session_id', float_format='%f')
!head -3 test_predictions_lr_kaggle.csv

/Users/eigenein/GitHub/mipt-ya-ml-spec/venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/eigenein/GitHub/mipt-ya-ml-spec/venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/eigenein/GitHub/mipt-ya-ml-spec/venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


[LibLinear]session_id,user_id
1,255
2,149


In [54]:
Path('answer6_6.txt').write_text('0.19891');